# In this part,DataFrame is used instead of RDD

In [5]:
# import moduels
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
import os
root=os.getcwd() # root is the current directory 

In [3]:
#  get the dictionary movieID-->movieName
def getMovieNames():
    movieNames = {}
    with open(os.path.join(root,"ml_100k/u.ITEM")) as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [6]:
# create a Spark Session, which can be used for DataFrame
spark = SparkSession.builder.config("spark.sql.warehouse.dir", os.path.join(root,'temp')).appName("PopularMovies").getOrCreate()
nameDic=getMovieNames()

In [10]:
# get the data from text file
lines = spark.sparkContext.textFile(os.path.join(root,'ml_100k/u.data'))
# map line to row, movieID as the key word
moviesRDD = lines.map(lambda x: Row(movieID =int(x.split()[1])))

In [11]:
# use createDataFrame function to generate DataFrame
movieDF = spark.createDataFrame(moviesRDD)

In [17]:
# rank the movies by popularity
topMovieIDs = movieDF.groupBy("movieID").count().orderBy("count", ascending=False).cache()
#show top 10
top10=topMovieIDs.take(10)
for p in top10:
    print('move_{} {} has {} ratings'.format(p[0],nameDic[p[0]],p[1]))

move_50 Star Wars (1977) has 583 ratings
move_258 Contact (1997) has 509 ratings
move_100 Fargo (1996) has 508 ratings
move_181 Return of the Jedi (1983) has 507 ratings
move_294 Liar Liar (1997) has 485 ratings
move_286 English Patient, The (1996) has 481 ratings
move_288 Scream (1996) has 478 ratings
move_1 Toy Story (1995) has 452 ratings
move_300 Air Force One (1997) has 431 ratings
move_121 Independence Day (ID4) (1996) has 429 ratings


In [18]:
spark.stop()